In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import anndata
import geopandas as gpd
import scanpy as sc
import visualization_helper # Custom helper functions 
from PIL import Image # Converting image

from tifffile import imread, imwrite
from csbdeep.utils import normalize
from stardist.models import StarDist2D
from shapely.geometry import Polygon, Point
from scipy import sparse
from matplotlib.colors import ListedColormap

%matplotlib inline
%config InlineBackend.figure_format = 'retina'



2025-04-15 15:11:54.117089: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:467] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1744755114.177211    1394 cuda_dnn.cc:8579] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1744755114.195592    1394 cuda_blas.cc:1407] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
W0000 00:00:1744755114.329139    1394 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking the same target more than once.
W0000 00:00:1744755114.329163    1394 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking the same target more than once.
W0000 00:00:1744755114.329164    1394 computation_placer.cc:177] computation placer alr

In [2]:
import tensorflow as tf

gpus = tf.config.list_physical_devices('GPU')
for gpu in gpus:
    details = tf.config.experimental.get_device_details(gpu)
    print(f"GPU: {gpu.name}")
    print(details)


GPU: /physical_device:GPU:0
{'compute_capability': (8, 6), 'device_name': 'NVIDIA GeForce RTX 3080 Ti'}


In [12]:
import subprocess

def print_gpu_memory():
    output = subprocess.check_output(['nvidia-smi']).decode('utf-8')
    print(output)

print_gpu_memory()

Tue Apr 15 15:33:12 2025       
+---------------------------------------------------------------------------------------+
| NVIDIA-SMI 545.23.08              Driver Version: 572.83       CUDA Version: 12.8     |
|-----------------------------------------+----------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |         Memory-Usage | GPU-Util  Compute M. |
|                                         |                      |               MIG M. |
|=========================================+======================+======================|
|   0  NVIDIA GeForce RTX 3080 Ti     On  | 00000000:2B:00.0  On |                  N/A |
|  0%   38C    P8              38W / 370W |  11462MiB / 12288MiB |     21%      Default |
|                                         |                      |                  N/A |
+-----------------------------------------+----------------------+--

In [5]:
def stardist_2D_versatile_he(img, nms_thresh=None, prob_thresh=None):
    # axis_norm = (0,1)   # normalize channels independently
    axis_norm = (0, 1, 2)  # normalize channels jointly
    # Make sure to normalize the input image beforehand or supply a normalizer to the prediction function.
    # this is the default normalizer noted in StarDist examples.
    img = normalize(img, 1, 99.8, axis=axis_norm)
    model = StarDist2D.from_pretrained("2D_versatile_he")
    labels, _ = model.predict_instances(
        img, nms_thresh=nms_thresh, prob_thresh=prob_thresh
    )
    return labels

In [11]:
dir_base = (
    '/mnt/c/Users/jonan/Documents/1Work/RoseLab/Spatial/'
    'dietary_droject/data/Rose_Li_VisiumHD/'
    'BANOSSM_SSM0015_1_PR_Whole_C1_VISHD_F07833_22WJCYLT3/outs/spatial/'
)


filename = 'tissue_hires_image.png'

img_pil = Image.open(dir_base + filename).convert("RGB")
img_np = np.array(img_pil)

# Normalize across all channels
img_norm = normalize(img_np, 1, 99.8, axis=(0, 1, 2))
print(img_np.shape)


(6000, 5656, 3)


In [10]:
from stardist.models import StarDist2D

model = StarDist2D.from_pretrained("2D_versatile_he")
help(model.predict_instances)

Found model '2D_versatile_he' for 'StarDist2D'.
Loading network weights from 'weights_best.h5'.
Loading thresholds from 'thresholds.json'.
Using default values: prob_thresh=0.692478, nms_thresh=0.3.
Help on method _predict_instances_generator in module stardist.models.base:

_predict_instances_generator(img, axes=None, normalizer=None, sparse=True, prob_thresh=None, nms_thresh=None, scale=None, n_tiles=None, show_tile_progress=True, verbose=False, return_labels=True, predict_kwargs=None, nms_kwargs=None, overlap_label=None, return_predict=False) method of stardist.models.model2d.StarDist2D instance
    Predict instance segmentation from input image.
    
    Parameters
    ----------
    img : :class:`numpy.ndarray`
        Input image
    axes : str or None
        Axes of the input ``img``.
        ``None`` denotes that axes of img are the same as denoted in the config.
    normalizer : :class:`csbdeep.data.Normalizer` or None
        (Optional) normalization of input image before pr

In [46]:
# Run StarDist segmentation
model = StarDist2D.from_pretrained("2D_versatile_he")
model_scale = 7
labels, _ = model.predict_instances(img_norm,               # Default values
                                    scale= model_scale,     # 1
                                    n_tiles = (20, 20, 1),  # 
                                    prob_thresh
                                    show_tile_progress=True)

Found model '2D_versatile_he' for 'StarDist2D'.
Loading network weights from 'weights_best.h5'.
Loading thresholds from 'thresholds.json'.
Using default values: prob_thresh=0.692478, nms_thresh=0.3.


  0%|                                                                                       | 0/400 [00:00<?, ?it/s]2025-04-15 16:59:47.184553: E external/local_xla/xla/stream_executor/cuda/cuda_timer.cc:86] Delay kernel timed out: measured time has sub-optimal accuracy. There may be a missing warmup execution, please investigate in Nsight Systems.
2025-04-15 16:59:47.528902: E external/local_xla/xla/stream_executor/cuda/cuda_timer.cc:86] Delay kernel timed out: measured time has sub-optimal accuracy. There may be a missing warmup execution, please investigate in Nsight Systems.
2025-04-15 16:59:47.633785: E external/local_xla/xla/service/slow_operation_alarm.cc:73] Trying algorithm eng13{} for conv %cudnn-conv-bias-activation.57 = (f32[1,32,2288,2176]{3,2,1,0}, u8[0]{0}) custom-call(f32[1,3,2288,2176]{3,2,1,0} %bitcast.689, f32[32,3,3,3]{3,2,1,0} %bitcast.696, f32[32]{0} %bitcast.698), window={size=3x3 pad=1_1x1_1}, dim_labels=bf01_oi01->bf01, custom_call_target="__cudnn$convBiasActiv

In [48]:
import matplotlib.pyplot as plt
import os

# Output directory
img_dir = (
    '/mnt/c/Users/jonan/Documents/1Work/RoseLab/'
    'Spatial/dietary_droject/figures/cell_segmentation/'
)
os.makedirs(img_dir, exist_ok=True)
filename = 'stardist_overlay_scale_' + str(model_scale) + '.png'
output_path = os.path.join(img_dir, filename)

# output_path = os.path.join(img_dir, 'stardist_overlay_scale_', model_scale, '.png')

# === Parameters to TUNE ===
segmentation_alpha = 0.4  # transparency of segmentation (0 = fully transparent, 1 = fully opaque)
cmap_segmentation = 'cividis'  # you can change this if you prefer other color schemes

# === Plot H&E background and segmentation overlay ===
plt.figure(figsize=(15, 15))
plt.imshow(img_np)  # show H&E image
plt.imshow(labels, cmap=cmap_segmentation, alpha=segmentation_alpha)  # overlay with alpha
plt.axis('off')
plt.title("StarDist Segmentation Over H&E")

# Save high-resolution overlay image
plt.savefig(output_path, dpi=300, bbox_inches='tight', pad_inches=0)
plt.close()

print(f"Overlay saved at:\n{output_path}")


Overlay saved at:
/mnt/c/Users/jonan/Documents/1Work/RoseLab/Spatial/dietary_droject/figures/cell_segmentation/stardist_overlay_scale_7.png


In [30]:
print(img_pil.size)  # returns (width, height)
print(img_np.shape) 

(5656, 6000)
(6000, 5656, 3)


In [8]:
dir_base + filename

'/mnt/c/Users/jonan/Documents/1Work/RoseLab/Spatial/dietary_droject/data/Rose_Li_VisiumHD/BANOSSM_SSM0015_1_PR_Whole_C1_VISHD_F07833_22WJCYLT3/outs/spatial/tissue_hires_image.png'